In [1]:
%load_ext autoreload
%autoreload 2
%matplotlib notebook

In [2]:
import torch
from experiments import intervention_utils
import os
from cem.data.CUB200.cub_loader import load_data, find_class_imbalance
from torchvision.models import resnet50
from cem.models.cem import ConceptEmbeddingModel
import pytorch_lightning as pl
import numpy as np
import cem.train.training as cem_train
import re
import random

In [12]:
CUB_DIR = 'CUB/'
BASE_DIR = os.path.join(CUB_DIR, 'images/CUB_200_2011/images')
num_workers=8
n_tasks = 200
n_concepts = 112
gpu = 0
sample_train = 0.1
concept_group_map = intervention_utils.CUB_CONCEPT_GROUP_MAP

In [4]:
og_config = dict(
        cv=5,
        max_epochs=300,
        patience=15,
        batch_size=128,
        num_workers=num_workers,
        emb_size=16,
        extra_dims=0,
        concept_loss_weight=5,
        normalize_loss=False,
        learning_rate=0.01,
        weight_decay=4e-05,
        weight_loss=True,
        pretrain_model=True,
        c_extractor_arch="resnet50",
        optimizer="sgd",
        bool=False,
        early_stopping_monitor="val_loss",
        early_stopping_mode="min",
        early_stopping_delta=0.0,
        sampling_percent=1,
        architecture="ConceptEmbeddingModel",
        momentum=0.9,
        shared_prob_gen=True,
        sigmoidal_prob=False,
        sigmoidal_embedding=False,
        training_intervention_prob=0.0,
        embeding_activation=None,
        concat_prob=False,
    )

In [5]:
train_data_path = os.path.join(CUB_DIR, 'preprocessed/train.pkl')
if og_config['weight_loss']:
    imbalance = find_class_imbalance(train_data_path, True)
else:
    imbalance = None

val_data_path = train_data_path.replace('train.pkl', 'val.pkl')
test_data_path = train_data_path.replace('train.pkl', 'test.pkl')

In [6]:
selected_concepts = np.arange(n_concepts)
def subsample_transform(sample):
    if isinstance(sample, list):
        sample = np.array(sample)
    return sample[selected_concepts]

In [8]:
train_dl = load_data(
    pkl_paths=[train_data_path],
    use_attr=True,
    no_img=False,
    batch_size=og_config['batch_size'],
    uncertain_label=False,
    n_class_attr=2,
    image_dir='images',
    resampling=False,
    root_dir='.',
    num_workers=og_config['num_workers'],
    concept_transform=subsample_transform,
    path_transform=lambda path: path.replace("CUB//",""),
)
val_dl = load_data(
    pkl_paths=[val_data_path],
    use_attr=True,
    no_img=False,
    batch_size=og_config['batch_size'],
    uncertain_label=False,
    n_class_attr=2,
    image_dir='images',
    resampling=False,
    root_dir='.',
    num_workers=og_config['num_workers'],
    concept_transform=subsample_transform,
    path_transform=lambda path: path.replace("CUB//",""))
test_dl = load_data(
    pkl_paths=[test_data_path],
    use_attr=True,
    no_img=False,
    batch_size=og_config['batch_size'],
    uncertain_label=False,
    n_class_attr=2,
    image_dir='images',
    resampling=False,
    root_dir='.',
    num_workers=og_config['num_workers'],
    concept_transform=subsample_transform,
    path_transform=lambda path: path.replace("CUB//","")
)

In [9]:
if sample_train < 1.0:
    train_dataset = train_dl.dataset
    train_size = round(len(train_dataset)*sample_train)
    train_subset = random.sample(range(0,len(train_dataset)),train_size)
    train_dataset = torch.utils.data.Subset(train_dataset, train_subset)
    sample_train_dl = torch.utils.data.DataLoader(train_dataset,batch_size=64, shuffle=True, 
                                           drop_last=True, num_workers=num_workers)



In [10]:
identity = lambda s: s
opposite = lambda s: 1-s
related_concepts = {}
for i in range(112):
    related_concepts[i] = {}
    for j in range(112):
        related_concepts[i][j] = (identity,1)

In [15]:
trainer = pl.Trainer(
            gpus=gpu,
        )

GPU available: False, used: False
TPU available: False, using: 0 TPU cores
IPU available: False, using: 0 IPUs


In [17]:
results_by_num_groups = {}

In [18]:
for num_groups_intervened in [0,6,12,18,24]:
    intervention_idxs = intervention_utils.random_int_policy(
                num_groups_intervened=num_groups_intervened,
                concept_group_map=concept_group_map,
                config=og_config,
            )
        
    model = intervention_utils.load_trained_model(
                config=og_config,
                n_tasks=n_tasks,
                n_concepts=n_concepts,
                result_dir="",
                split=0,
                imbalance=None,
                intervention_idxs=intervention_idxs,
                train_dl=sample_train_dl,
                sequential=False,
                independent=False,
        related_concepts=related_concepts
            )
    [test_results] = trainer.test(model, val_dl, verbose=False,)
    results_by_num_groups[num_groups] = test_results

/rds/user/njr61/hpc-work/anaconda3/envs/cem/lib/python3.7/site-packages/torchvision/models/_utils.py:209: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  f"The parameter '{pretrained_param}' is deprecated since 0.13 and may be removed in the future, "
/rds/user/njr61/hpc-work/anaconda3/envs/cem/lib/python3.7/site-packages/torchvision/models/_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=ResNet50_Weights.IMAGENET1K_V1`. You can also use `weights=ResNet50_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)


Testing: 0it [00:00, ?it/s]

NameError: name 'num_groups' is not defined

In [ ]:
test_results